# 課題1 WAVデータの入出力，STFT
.wav 形式のデータをPython上で読み込み，書き込む  
STFTと簡単なハイパスフィルタの作成，逆変換による波形復元

## 必要なライブラリのインポートおよび雑多な変数の設定
おまじない

In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import soundfile as sf
import scipy.signal
import os


eps = 1e-10     ### 「めっちゃ小さい数」として使う便利な値
plt.rcParams.update({
    ####### matplotlibでグラフを出力するときの設定
    'font.size' : 18,     ### フォントサイズ
    'font.family': ['Times New Roman']   ### フォントの種類
})


## 課題1.1 読み込むファイルの設定（以下のコードに書き込む）
書き出すファイル名を参考に読み込むファイル名を記述

In [ ]:
####### WRITE HERE ######################################################
####### 読み込むファイル名
input_file = "wav_data" + os.sep + "input" + os.sep + "saudade.wav"
#########################################################################

####### 書き出すファイル名
output_file = "wav_data" + os.sep + "output" + os.sep + "out1_1.wav"

wave, sr = librosa.core.load(input_file)  ### load input wav
sf.write(output_file, wave, sr)     ### save output wav
print(f"waveのサイズ:{wave.shape}\nサンプリングレート{sr}")

## matplotlibで波形の表示

In [ ]:
t = np.arange(0, wave.shape[0]/sr, 1/sr)  ### x axis [sec.]
plt.plot(t, wave)           ### x軸: t, y軸: wave
plt.xlabel("Time [sec.]")   ### x軸ラベル
plt.ylabel("Amplitude")     ### y軸ラベル
plt.ylim([-1, 1])           ### y軸表示範囲
plt.tight_layout()          ### 図のサイズをなんかいい感じにする
plt.show()                  ### グラフの表示


## 課題1.2 waveの切り出し（以下のコードに書き込む）

In [ ]:
duration = 5    ### この課題で扱う曲の長さ[sec.]

####### WRITE HERE ######################################################
####### duration秒切り出すために必要なサンプル数を計算
dur_length = duration*sr
#########################################################################

t2 = np.arange(0, duration, 1/sr)  ## x axis [sec.]

####### WRITE HERE ######################################################
####### waveの0~dur_length分のデータをwave2に代入
####### あるいは「スライス表記」を用いるとfor文なしでも書ける
# for i in range(dur_length):

wave2 = wave[:dur_length]

####### 「matplotlibで波形の表示」のコードを参考にwave2のグラフを描画
plt.plot(t2, wave2)
plt.xlabel("Time [sec.]")
plt.ylabel("Amplitude")
plt.ylim([-1, 1])
plt.tight_layout()
plt.show()

####### 「課題1.1」を参考にwave2をout1_2.wavとして書き出し
### この先何度も書くのめんどいのでこれだけ変数化
output_dir = "wav_data" + os.sep + "output" + os.sep    


#########################################################################

## STFTによるスペクトログラム表示

In [ ]:
x = librosa.core.stft(wave)  ### STFT
x_abs = np.abs(x)  ### 振幅成分の取り出し
x_abs_db = librosa.core.amplitude_to_db(x_abs)  ### db単位へ変換
librosa.display.specshow(x_abs_db, cmap='jet', sr=sr, x_axis='time', y_axis='log')  ### スペクトログラムを表示(y軸を対数軸に変換)
plt.colorbar(format='%+2.0f dB')
plt.tight_layout()
plt.show()

print(f"STFT後のサイズ:縦{x.shape[0]}，横{x.shape[1]}")

## 課題1.3 wave2のスペクトログラム

In [ ]:
####### WRITE HERE ######################################################
####### 「STFTによるスペクトログラム表示」を参考にwave2のスペクトログラムを描画
x2 = librosa.core.stft(wave2)  ### STFT
x_abs2 = np.abs(x2)  ### 振幅成分の取り出し
x_abs_db2 = librosa.core.amplitude_to_db(x_abs2)  ### db単位へ変換
librosa.display.specshow(x_abs_db2, cmap='jet', sr=sr, x_axis='time', y_axis='log')  ### スペクトログラムを表示(y軸を対数軸に変換)
plt.colorbar(format='%+2.0f dB')
plt.tight_layout()
plt.show()
#########################################################################

print(f"STFT後のサイズ:縦(周波数){x2.shape[0]}，横(時刻){x2.shape[1]}")

## 課題1.4 サンプリングレートの変更
サンプリングレートsr \[Hz\]の場合，  
スペクトログラムは0 ~ sr/2 \[Hz\]まで表示(サンプリング定理)  
サンプリングレートを半分にすると，表示できる最大周波数が半分になる．

In [ ]:
####### サンプリングレート半分
sr3 = sr//2
####### x2のサイズを参考にx3のサイズを決める
x3 = np.zeros((x2.shape[0]//2+1, x2.shape[1]))

print(f"x3のサイズ:縦(周波数){x3.shape[0]}，横(時刻){x3.shape[1]}")

####### WRITE HERE ######################################################
####### x3の各要素にx2を代入
for i in range(x3.shape[0]):
    for j in range(x3.shape[1]):
        x3[i, j] = x2[i, j]     

####### あるいは「スライス表記」を用いて↑の3行分を1行で書いても可


#########################################################################

####### x3のスペクトログラムを表示
x_abs3 = np.abs(x3)  ### 振幅成分の取り出し
x_abs_db3 = librosa.core.amplitude_to_db(x_abs3)  ### db単位へ変換

####### ※srとして指定する値に注意する
librosa.display.specshow(x_abs_db3, cmap='jet', sr=sr3, x_axis='time', y_axis='log')  ### スペクトログラムを表示(y軸を対数軸に変換)
plt.colorbar(format='%+2.0f dB')
plt.tight_layout()
plt.show()
   

## 課題1.5 ハイパスフィルタ
1000 Hzくらいより大きい周波数のみ通し，それ以下は0にしたい．  
STFT後の縦サイズによって0~sr/2までの周波数が等分されていることから，  
縦サイズの\[  \]番目以降を残し，それ以下の部分を0にすればよい

In [ ]:
x4 = x2.copy()

####### ↑の[  ]に入る数字を入れる（四捨五入した整数値）
cut_index = 93

####### x4の該当する要素を0にする（for文あるいはスライス表記）
x4[:cut_index] = 0

####### x4のスペクトログラムを表示
x_abs4 = np.abs(x4)  ### 振幅成分の取り出し
x_abs_db4 = librosa.core.amplitude_to_db(x_abs4)  ### db単位へ変換
librosa.display.specshow(x_abs_db4, cmap='jet', sr=sr, x_axis='time', y_axis='log')  ### スペクトログラムを表示(y軸を対数軸に変換)
plt.colorbar(format='%+2.0f dB')
plt.tight_layout()
plt.show()


## 逆変換(iSTFT)により波形成分を得る

In [ ]:
####### iSTFTにより波形に戻す
wave3 = librosa.core.istft(x)

####### wave3の表示
t3 = np.arange(0, wave3.shape[0]/sr, 1/sr)  ## x axis [sec.]
plt.plot(t3, wave3)
plt.xlabel("Time [sec.]")
plt.ylabel("Amplitude")
plt.ylim([-1, 1])
plt.tight_layout()
plt.show()

####### wave3をout1_3.wavとして保存
sf.write(output_dir + os.sep + "out1_3.wav", wave3, sr)

## 課題1.6 x2, x3, x4にiSTFTを施し波形を得る

In [ ]:
####### WRITE HERE ######################################################
####### 「逆変換(iSTFT)により波形成分を得る」のコードを参考にx2をwave4へiSTFT
wave4 = librosa.core.istft(x2)

####### wave4の波形を表示
t4 = np.arange(0, wave4.shape[0]/sr, 1/sr)  ## x axis [sec.]
plt.plot(t4, wave4)
plt.xlabel("Time [sec.]")
plt.ylabel("Amplitude")
plt.ylim([-1, 1])
plt.tight_layout()
plt.show()

####### wave4をout1_4.wavとして保存
sf.write(output_dir + os.sep + "out1_4.wav", wave4, sr)

#########################################################################

In [ ]:
####### WRITE HERE ######################################################
####### x3をwave5へiSTFT
wave5 = librosa.core.istft(x3)

####### wave4の波形を表示（※サンプリングレートに注意！）
t5 = np.arange(0, wave5.shape[0]/sr3, 1/sr3)  ## x axis [sec.]
plt.plot(t5, wave5)
plt.xlabel("Time [sec.]")
plt.ylabel("Amplitude")
plt.ylim([-1, 1])
plt.tight_layout()
plt.show()

####### wave5をout1_5.wavとして保存（※サンプリングレートに注意！）
sf.write(output_dir + os.sep + "out1_5.wav", wave5, sr3)


#########################################################################

In [ ]:
####### WRITE HERE ######################################################
####### x4をwave6へiSTFT
wave6 = librosa.core.istft(x4)


####### wave6の波形を表示
t6 = np.arange(0, wave6.shape[0]/sr, 1/sr)  ## x axis [sec.]
plt.plot(t6, wave6)
plt.xlabel("Time [sec.]")
plt.ylabel("Amplitude")
plt.ylim([-1, 1])
plt.tight_layout()
plt.show()

####### wave6をout1_6.wavとして保存
sf.write(output_dir + os.sep + "out1_6.wav", wave6, sr)


#########################################################################